In [28]:
import urllib.request
import re
import requests
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import json
import time
from networkx.algorithms.community.centrality import girvan_newman
from fa2 import ForceAtlas2

In [2]:
def imdbId_to_wikiLink(imdbId):

    imdbId = str(imdbId)
    
    while len(imdbId)<7:
        imdbId = str(0) + imdbId  

    url = 'https://query.wikidata.org/sparql'
    query = '''
    SELECT ?wppage WHERE {                                                          
    ?subject wdt:P345 'tt''' + imdbId + '''' .                                                   
      ?wppage schema:about ?subject .                                               
      FILTER(contains(str(?wppage),'//en.wikipedia'))                               
    }
    '''
    r = requests.get(url, params = {'format': 'json', 'query': query})
    while (r.status_code == 429):
        time.sleep(0.1)
        r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
    if (len(data['results']['bindings']) == 0):
        return None
        
    weblink = data['results']['bindings'][0]['wppage']['value']

    
    wikilink = ''
    
    #match the last /wikilink part of url
    regex = re.compile(r".+\/(.+)")
    wikimatch = re.search(regex, weblink)
    
    if wikimatch is not None:
        wikilink = wikimatch.group(1)
    
    return wikilink

In [3]:
def wikiLink_to_wikiText(wikiLink):

    #as described at https://www.mediawiki.org/wiki/API:Tutorial
    baseurl = "https://en.wikipedia.org/w/api.php?"
    action = "action=query"
    content = "prop=revisions&rvprop=content"
    dataformat ="format=json"
    
    title = "titles={}".format(wikilink)
    query = "{}{}&{}&{}&{}".format(baseurl, action, content, title, dataformat)
    wikiresponse = urllib.request.urlopen(query)
    wikidata = wikiresponse.read()
    wikitext = wikidata.decode('utf-8')
    
    return wikitext

In [4]:
def genre_in_common(movie1, movie2):
    
    #define the to list of genres
    G1 = G.nodes[movie1]['genres']
    G2 = G.nodes[movie2]['genres']
    
    #make list of genres in common
    common = [i for i in G1 if i in G2]
    
    if (len(common)>0):
        return True
    return False

In [5]:
def suggestMovie(movieId):
    
    #used to keep track of strongest link to other movie
    max_weight = 0
    
    for edge in G.edges(movieId):
        
        #if link is the strongest we've seen, then save it
        if G[edge[0]][edge[1]]['weight'] > max_weight:
            max_weight = G[edge[0]][edge[1]]['weight']
            recommended_edge = edge
    
    #return the recommended movie from the strongest link
    if recommended_edge[0] != movieId:
        recommended_movie = recommended_edge[0]
    else:
        recommended_movie = recommended_edge[1]
        
    return G.nodes[recommended_movie]['title']  

In [6]:
def get_yearly_avg(year):
    
    #used to calculate average
    count = 0
    sum = 0
    
    for movie in G.nodes:
        
        #match numbers in ()
        match = re.search(r"\(([0-9]*?)\)", G.nodes[movie]['title'])
        
        if (match is not None):
            
            #if movie is from year, update count and sum
            if (int(match.group(1)) == year):
                count += 1
                sum += G.nodes[movie]['avg_rating']
                
    if (count == 0):
        return 0
    return (sum/count)

In [7]:
def movieId_to_filetitle(movieId):
    illegal = ['#','%','&','{','}','\\','$','!','\'','"',':','<','>','*','?','/',' ','+','`','|','=']

    fileTitle = G.nodes[movieId]['title'] + '.txt'
    
    for sign in illegal:
        fileTitle = fileTitle.replace(sign,'_')
    return fileTitle

In [8]:
readerLinks = pd.read_csv('links.csv')
readerMovies = pd.read_csv('movies.csv')
readerRatings = pd.read_csv('ratings.csv')

In [9]:
links = []
for i in range(len(readerLinks)):

    movieId = readerLinks.iloc[i,0]
    imdbId = readerLinks.iloc[i,1]
    
    links.append([movieId, imdbId])

In [10]:
movies = []
for i in range(len(readerMovies)):

    movieId = readerMovies.iloc[i,0]
    titel = readerMovies.iloc[i,1]
    genres = readerMovies.iloc[i,2]
    
    movies.append([movieId, titel, genres])

In [11]:
ratings = []
for i in range(len(readerRatings)):

    userId = readerRatings.iloc[i,0]
    movieId = readerRatings.iloc[i,1]
    rating = readerRatings.iloc[i,2]
    
    ratings.append([userId, movieId, rating])

In [163]:
G = nx.Graph()

In [164]:
for movie in movies:
    
    #make list of genres from string of genres
    genres = re.findall(r"([^|]+)",movie[2])
    
    G.add_node(movie[0], title=movie[1], avg_rating=0, rating_count=0, genres=genres,)

In [165]:
#user likes everything rated limit_for_love or higher

limit_for_love = 4

#for every movie loved by user, make a link/increase weight of existing link, to movies also loved by user
for i in range(len(ratings)):
    
    #defining movie
    movie = ratings[i][1]
    
    #update average rating of movie
    old_average = G.nodes[movie]['avg_rating']
    old_count = G.nodes[movie]['rating_count']
    
    G.nodes[movie]['rating_count'] += 1
    G.nodes[movie]['avg_rating'] = (old_average*old_count + ratings[i][2])/G.nodes[movie]['rating_count']
    
    #if user doesnt love movie dont bother linking to it
    if (ratings[i][2] < limit_for_love):
        continue
    
    #increase index to avoid linking to itself
    j = i+1
    
    #avoid out of range
    if(j < len(ratings)):
        
        #look at all other ratings made by same user
        while (ratings[j][0] == ratings[i][0]):
            
            second_movie = ratings[j][1]
            
            #if the user loves second_movie and the two movies share a genre, add an edge between them or increase weight of existing
            if (ratings[j][2] >= limit_for_love and genre_in_common(movie, second_movie)):
                if (G.has_edge(movie, second_movie)):
                    G[movie][second_movie]['weight'] += 1 #increase weight on existing edge
                else:
                    G.add_edge(movie, ratings[j][1], weight=1) #or create new edge
            j+=1
            if (j>=len(ratings)):
                break

In [166]:
len(G.nodes)

9742

In [167]:
len(G.edges)

1664844

In [168]:
degrees = [G.degree(n) for n in G.nodes()]

hist = plt.hist(degrees, edgecolor='blue', alpha=0.5,align='left', bins=range(0,max(degrees)), label='movies')

plt.legend(loc='upper right')
plt.title('Degree')
plt.xlabel("Degree")
plt.ylabel("Frequency")

plt.show()

TypeError: 'int' object is not callable

In [ ]:
x = []
y = []

for i in range(1915,2019):
    y.append(get_yearly_avg(i))
    x.append(i)
  
plt.plot(x, y)
  
plt.xlabel('year')
plt.ylabel('average rating')
  
plt.title('Yearly average movie ratings')
  
plt.show()

In [ ]:
#try out suggestMovie:
#Indiana Jones and the Raiders of the Lost Ark (id = 1198)
#Toy Story (id = 1)
#Harry Potter and the Sorcerer's Stone (id = 4896)
#Avatar (id = 72998)
#Sinister (id = 97188)

suggestMovie(4896)

In [ ]:
#downloading wikitexts
#movie_txt = []

#for link in links:
    
#    movieId = link[0]
#    imdbId = link[1]
    
#    fileTitle = movieId_to_filetitle(movieId)
    
#    wikilink = imdbId_to_wikiLink(imdbId)
#    if wikilink is None:
#        continue
    
#    wikitext = wikiLink_to_wikiText(wikilink)
    
#    f = open('wikitexts/' + fileTitle, 'w')
#    f.write(json.dumps(json.loads(wikitext)))   

In [ ]:
x = []
y = []
for i in range(51):
    loved = 0
    for node in G.nodes:
        if (G.nodes[node]['avg_rating'] >= limit_for_love and G.nodes[node]['rating_count'] > i):
            loved += 1
    x.append(i)
    y.append(loved)

plt.plot(x, y)
  
plt.xlabel('minimum ratings')
plt.ylabel('count of loved movies')
  
plt.title('How many ratings does a loved movie have?')
  
plt.show()

In [ ]:
#keep only strongest link

#for node in G.nodes:    
#    max = 0
#    to_remove = []
#    
#    for edge in G.edges(node):
#        if G[edge[0]][edge[1]]["weight"] > max:
#            max = G[edge[0]][edge[1]]["weight"]
#
#    for edge in G.edges(node):
#        if G[edge[0]][edge[1]]["weight"] != max:
#            to_remove.append(edge)
#        
#    G.remove_edges_from(to_remove)

In [169]:
#weakly_connected_components only works on undirected graphs
G_directed = G.to_directed()
Gsub = max(nx.weakly_connected_components(G_directed))
Gsub = nx.subgraph(G, Gsub)
Gsub = Gsub.to_undirected()

TypeError: 'int' object is not callable

In [ ]:
defined_colors = []

red = "#A83432"
blue = "#323EA8"

for node in Gsub:
    #color movies liked on average red
    if G.nodes[node]['avg_rating'] >= limit_for_love and G.nodes[node]['rating_count'] > 20:
        defined_colors.append(red)
    #color the rest blue
    else:
        defined_colors.append(blue)

In [ ]:
forceatlas2 = ForceAtlas2(gravity = 1.0)

In [ ]:
positions = forceatlas2.forceatlas2_networkx_layout(Gsub, pos=None, iterations= 100)

In [ ]:
fig = plt.figure(1, figsize=(200, 100), dpi=60)
nodes = nx.draw_networkx_nodes(Gsub, positions, alpha=1, node_color = defined_colors)
nx.draw_networkx_edges(Gsub, positions, alpha=0.5)